In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import matplotlib.pyplot as plt
import seaborn as sns 

# Введение (introduction)

Перед нами ежедневные исторические данные о продажах. 
Задача состоит в том, чтобы спрогнозировать общее количество продуктов, проданных в каждом магазине для тестового набора. Список магазинов и продуктов немного меняется каждый месяц - создание надежной модели, способной справиться с такими ситуациями, является частью задачи.

**File descriptions**
* sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
* test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
* 
* items.csv - дополнительная информация о товарах / продуктах.
* item_categories.csv  - дополнительная информация о категориях товаров.
* shops.csv- дополнительная информация о магазинах

**Data fields**
* ID - an Id that represents a (Shop, Item) tuple within the test set
* shop_id - unique identifier of a shop
* item_id - unique identifier of a product
* item_category_id - unique identifier of item category
* item_cnt_day - number of products sold (мой прогноз - сумма этой переменной) 
* item_price - current price of an item
* date - date in format dd/mm/yyyy
* date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
* item_name - name of item
* shop_name - name of shop
* item_category_name - name of item category

***Оценка качества модели будет производиться root mean squared error (RMSE)***

# Загрузка данных

Загружаем библиотеки и читаем наборы данных для исследовательского анализа и моделирования.
Посмотрим, какую информацию передают эти данные.

In [ ]:
train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
categories = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
submission = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
submission.head()

Как мы видим, тестовый набор отличается по размеру и структуре по сравнению с обучающим набором. 
У нас есть функции "shop_id" и "item_id" в тестовом наборе, которые также присутствуют в обучающем наборе. 
Каждое наблюдение в тестовом наборе имеет связанный с ним идентификатор. Если мы посмотрим на наш файл отправки, нам нужно будет отправить ежемесячный счет (item_cnt_month) для этого конкретного идентификатора. Это означает, что нам нужно предсказать число для ежемесячного количества продажи конкретного товара в конкретном магазине.

# Визуальный анализ данных

Давайте посмотрим на распределение обучающего и тестового набора, чтобы лучше понять наш набор данных и проверить его на совпадение или отсутствие.

In [ ]:
fig = plt.figure(figsize=(18,9))
plt.subplots_adjust(hspace=.5)

plt.subplot2grid((3,3), (0,0), colspan = 3)
train['shop_id'].value_counts(normalize=True).plot(kind='bar', alpha=0.7)
plt.title('Shop ID Values in the Training Set (Normalized)')

plt.subplot2grid((3,3), (1,0))
train['item_id'].plot(kind='hist', alpha=0.7)
plt.title('Item ID Histogram')

plt.subplot2grid((3,3), (1,1))
train['item_price'].plot(kind='hist', alpha=0.7, color='orange')
plt.title('Item Price Histogram')

plt.subplot2grid((3,3), (1,2))
train['item_cnt_day'].plot(kind='hist', alpha=0.7, color='green')
plt.title('Item Count Day Histogram')

plt.subplot2grid((3,3), (2,0), colspan = 3)
train['date_block_num'].value_counts(normalize=True).plot(kind='bar', alpha=0.7)
plt.title('Month (date_block_num) Values in the Training Set (Normalized)')

plt.show()

Приведенные выше графики являются хорошим способом взглянуть на необработанное распределение тестового набора данных. Вот некоторые наблюдения:

* У нас есть 60 'shop_id', но в наборе данных они распределены неравномерно. Четыре (<7%) из этих магазинов составляют ~25% этого набора данных. Это магазины (31, 25, 54, 28).

* Идентификаторы элементов, по-видимому, имеют вариации по частоте. Пока нельзя увидеть этому причину, но мы можем исследовать это дальше. Некоторые категории товаров обязательно будут продаваться лучше и, возможно, товары одной и той же категории будут ближе друг к другу в том, что касается их распределения идентификаторов

* Из огромных пустых пространств в гистограммах "item_price" и "item_cnt_day" мы можем сделать вывод, что в их распределении есть выбросы. Давайте напишем ниже простой код, чтобы определить значение этих выбросов.

* Построив график отдельных месяцев с января 2013 года по октябрь 2015 года можно заметить, что блок 12 месяца, соответствующий декабрю 2013 года, имел наибольшее количество продаж. 23-й месяц, который соответствует декабрю 2014 года, имел второе по величине количество продаж. Ниже будем использовать несколько лучших графиков для наблюдения за ежемесячными тенденциями продаж.

In [ ]:
sns.boxplot(x=train.item_cnt_day)

In [ ]:
sns.boxplot(x=train.item_price)

# Подготовка данных (Data preporation)

**1. Работа с выбросами и их исключение из данных**

In [ ]:
train['item_id'].value_counts(ascending=False)[:5]

Взглянув на товар id 20949, который был продан больше всего раз, можно сказать, что это "пакет"

In [ ]:
items.loc[items['item_id']==20949]

In [ ]:
categories.loc[categories['item_category_id']==71]

In [ ]:
test.loc[test['item_id']==20949].head(5)

In [ ]:
train['item_cnt_day'].sort_values(ascending=False)[:5]

In [ ]:
train[train['item_cnt_day'] == 2169]

Товар ID 11373 был продан 2169 раз в магазине 12 в один октябрьский день. 
Давайте еще немного осмотрим этот выброс.

In [ ]:
items[items['item_id'] == 11373]

Посмотрим на другие дневные продажи этого товара

In [ ]:
train[train['item_id'] == 11373].head(5)

Мы видим, что товар с ID 11373 обычно продается значительно меньше. 

Я вычислила, что медиана его значения 'item_cnt_day' равна 4. Это подтверждает, что высокое значение 2169 является аномалией, и мы должны избавиться от него.

In [ ]:
train = train[train['item_cnt_day'] < 800]

In [ ]:
train['item_price'].sort_values(ascending=False)[:5]

Еще один выброс.

На этот раз из "item_price". Рассмотрим его прежде чем удалять его.

In [ ]:
train[train['item_price'] == 307980]

In [ ]:
items[items['item_id'] == 6066]

Это антивирус, проданный 522 людям, и цена, вероятно, равна стоимости одной установки, умноженной на 522. 
Посмотрим, есть ли в нашем обучающем наборе другие транзакции, связанные с этим программным обеспечением.

In [ ]:
train[train['item_id'] == 6066]

Нет, а значит это Выброс - единственная транзакция. 
Вполне оправданно убрать это наблюдение из тренировочного набора.

In [ ]:
train = train[train['item_price'] < 100000]

Теперь проверим поле "item_price" на наличие выбросов по низким ценам

In [ ]:
train['item_price'].sort_values()[:5]

Существует значение -1. Это ошибка и мы должны проверить это наблюдение дальше

In [ ]:
train[train['item_price'] == -1]

Есть ли другие наблюдения для этого предмета (2973), которые могут помочь нам определить его цену

In [ ]:
train[train['item_id'] == 2973].head(5)

Действительно, этот предмет, похоже, был продан по довольно высокой цене, поэтому его значение -1 должно быть изменено. Мы должны заменить его медианой его цены, но это должно быть рассчитано для магазина (ID 12), для которого этот выброс существует. Если есть другие продажи этого товара в этом магазине за тот же месяц (date_block_num 4), то мы должны рассчитать медиану, используя это

In [ ]:
price_correction = train[(train['shop_id'] == 32) & (train['item_id'] == 2973) & (train['date_block_num'] == 4) & (train['item_price'] > 0)].item_price.median()
train.loc[train['item_price'] < 0, 'item_price'] = price_correction

**2. Распределение Тестовых Наборов**

Теперь давайте проведем анализ распределения тестового набора и посмотрим, сможем ли мы обнаружить какие-либо различия

In [ ]:
fig = plt.figure(figsize=(18,8))
plt.subplots_adjust(hspace=.5)

plt.subplot2grid((3,3), (0,0), colspan = 3)
test['shop_id'].value_counts(normalize=True).plot(kind='bar', alpha=0.7)
plt.title('Shop ID Values in the Test Set (Normalized)')

plt.subplot2grid((3,3), (1,0))
test['item_id'].plot(kind='hist', alpha=0.7)
plt.title('Item ID Histogram - Test Set')

plt.show()

1. Идентификаторы магазина распределены равномерно, в отличие от тренировочного набора. Размер шрифта меток быстро говорит, что в тестовом наборе отсутствуют определенные идентификаторы магазина, так как бары в графике обучающего набора "shop_id" были более плотно упакованы.
2. В то время как идентификаторы элементов на гистограмме сгруппированы, пики в тестовом наборе меньше. Тестовый набор гораздо меньше по форме, чем обучающий, и, естественно, значения частоты значительно ниже. Трудно сделать больше выводов из этой гистограммы.
Похоже, что некоторые значения shop_id и item_id полностью отсутствуют в тестовом наборе. Давайте посмотрим поближе и добавим некоторые цифры или проценты к этим отсутствующим значениям.

**3. Анализ магазинов**

In [ ]:
shops_train = train['shop_id'].nunique()
shops_test = test['shop_id'].nunique()
print('Shops in Training Set: ', shops_train)
print('Shops in Test Set: ', shops_test)

Однако это не означает, что обучающий набор содержит все магазины, присутствующие в тестовом наборе. Для этого нам нужно посмотреть, присутствует ли каждый элемент тестового набора в обучающем наборе. Давайте напишем простой код, чтобы увидеть, является ли список тестовых наборов подмножеством списка обучающих наборов

In [ ]:
shops_train_list = list(train['shop_id'].unique())
shops_test_list = list(test['shop_id'].unique())

flag = 0
if(set(shops_test_list).issubset(set(shops_train_list))): 
    flag = 1
      
if (flag) : 
    print ("Да, список-это подмножество других.") 
else : 
    print ("Нет, список-это не подмножество других.") 

**Удаляем дубликаты магазинов**

In [ ]:
shops

In [ ]:
train.loc[train['shop_id'] == 0, 'shop_id'] = 57
test.loc[test['shop_id'] == 0, 'shop_id'] = 57

train.loc[train['shop_id'] == 1, 'shop_id'] = 58
test.loc[test['shop_id'] == 1, 'shop_id'] = 58

train.loc[train['shop_id'] == 10, 'shop_id'] = 11
test.loc[test['shop_id'] == 10, 'shop_id'] = 11

train.loc[train['shop_id'] == 40, 'shop_id'] = 39
test.loc[test['shop_id'] == 40, 'shop_id'] = 39

train.loc[train['shop_id'] == 23, 'shop_id'] = 24
test.loc[test['shop_id'] == 23, 'shop_id'] = 24

**Извлечение города из названия магазина**

In [ ]:
cities = shops['shop_name'].str.split(' ').map(lambda row: row[0])
cities.unique()

In [ ]:
shops['city'] = shops['shop_name'].str.split(' ').map(lambda row: row[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit_transform(shops['city'])

Теперь просто добавим эти коды в фрейм данных магазинов. 
Мы можем избавиться от 'shop_name' и 'city' сохраняя 'shop_id' и 'city_label'

In [ ]:
shops['city_label'] = le.fit_transform(shops['city'])
shops.drop(['shop_name', 'city'], axis = 1, inplace = True)
shops.head()

**4. Анализ товаров**

In [ ]:
items_train = train['item_id'].nunique()
items_test = test['item_id'].nunique()
print('Items in Training Set: ', items_train)
print('Items in Test Set: ', items_test)

В обучающем наборе гораздо больше элементов, чем в тестовом, но это не означает, что обучающий набор содержит все элементы тестового набора. Для этого нам нужно посмотреть, присутствует ли каждый элемент тестового набора в обучающем наборе.

In [ ]:
items_train_list = list(train['item_id'].unique())
items_test_list = list(test['item_id'].unique())

flag = 0
if(set(items_test_list).issubset(set(items_train_list))): 
    flag = 1
      
if (flag) : 
    print ("Да, список-это подмножество других.") 
else : 
    print ("Нет, список-это не подмножество других.") 

Это означает, что есть определенные элементы, которые присутствуют в тестовом наборе, но полностью отсутствуют в тренировочном наборе. Попробуем это исправить

In [ ]:
len(set(items_test_list).difference(items_train_list))

Есть 363 элемента, которые присутствуют в тестовом наборе, но полностью отсутствуют в тренировочном наборе. Это не означает, что прогноз продаж по этим товарам должен быть нулевым, поскольку новые товары могут быть добавлены на рынок, или мы просто не располагали данными по этим товарам раньше. Однако возникают вопросы: как их предсказывать?

Прежде чем мы это сделаем, давайте узнаем больше о 5100 элементах в тестовом наборе. К каким категориям они относятся? Для каких категорий мы не должны делать прогнозы в тестовом наборе

In [ ]:
categories_in_test = items.loc[items['item_id'].isin(sorted(test['item_id'].unique()))].item_category_id.unique()
categories.loc[~categories['item_category_id'].isin(categories_in_test)]

**Группировка общих категорий и извлечение подкатегорий**

In [ ]:
le = preprocessing.LabelEncoder()

main_categories = categories['item_category_name'].str.split('-')
categories['main_category_id'] = main_categories.map(lambda row: row[0].strip())
categories['main_category_id'] = le.fit_transform(categories['main_category_id'])

categories['sub_category_id'] = main_categories.map(lambda row: row[1].strip() if len(row) > 1 else row[0].strip())
categories['sub_category_id'] = le.fit_transform(categories['sub_category_id'])

In [ ]:
categories.head()

Tеперь произведем обработку **'date'** для создания различных столбцов, связанных с датой, которые могут дать нам большое представление о данных.

In [ ]:
import datetime

train['date'] = pd.to_datetime(train['date'],format = '%d.%m.%Y')
print('Min date from train set: %s' % train['date'].min().date())
print('Max date from train set: %s' % train['date'].max().date())

In [ ]:
# print min and max num assigned to the months
print('Min date_block_num from train set: %s' % train['date_block_num'].min())
print('Max date_block_num from train set: %s' % train['date_block_num'].max())

Обучающие данные находятся в периоде с января 2013 года (date_block_num 0) по октябрь 2015 года (date_block_num 33).
Мы должны использовать запись продаж за последние 34 месяца, чтобы предсказать 35-й месяц, который является ноябрь 2015 (date_block_num 34).

Так как ответом в задании Predict Future Sales является предсказание совокупной суммы продаж в каждом магазине в Ноябре 2015 на тестовой выборке, найдем количество продаж в месяц на магазин

In [ ]:
from itertools import product

In [ ]:
# Тестирование генерации декартового произведения за январь 2013 года
shops_in_jan = train.loc[train['date_block_num']==0, 'shop_id'].unique()
items_in_jan = train.loc[train['date_block_num']==0, 'item_id'].unique()
jan = list(product(*[shops_in_jan, items_in_jan, [0]]))
print(len(jan))

Как видно, январь 2013 года содержит 357060 пересечений магазинов и предметов. 
Большинство из них не будут иметь продаж, и мы можем проверить это, как только проверим наш обучающий набор, который был сгруппирован по месяцам, чтобы увидеть, какие комбинации "товаров" и "магазинов" имеют количество продаж, связанных с ними.

Перед этим нам нужно сгенерировать это декартово произведение за все 33 месяца в обучающем наборе. Прежде чем сгенерировать его для всех месяцев, я сгенерирую его для февраля 2013 года, объединю его с январем 2013 года и создам фрейм данных.

In [ ]:
shops_in_feb = train.loc[train['date_block_num']==1, 'shop_id'].unique()
items_in_feb = train.loc[train['date_block_num']==1, 'item_id'].unique()
feb = list(product(*[shops_in_feb, items_in_feb, [1]]))
print(len(feb))

In [ ]:
cartesian_test = []
cartesian_test.append(np.array(jan))
cartesian_test.append(np.array(feb))
cartesian_test

При попытке создать фрейм данных из списков января и февраля закончилась память. Хитрость заключалась в том, чтобы преобразовать списки в массив numpy. Удобный метод numpy 'vstack' сформирует объект cartesian_test правильным образом, так что мы сможем преобразовать его в длинный фрейм данных формы.

In [ ]:
cartesian_test = np.vstack(cartesian_test)

In [ ]:
cartesian_test_df = pd.DataFrame(cartesian_test, columns = ['shop_id', 'item_id', 'date_block_num'])
cartesian_test_df.head()

In [ ]:
cartesian_test_df.shape

Тест прошел успешно - все работает и не "кушает" память. Теперь напишем небольшой скрипт для остальных месяцев

In [ ]:
from tqdm import tqdm_notebook

def downcast_dtypes(df):
    '''
        Меняем типы столбцов в датафрейме: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float16)
    df[int_cols]   = df[int_cols].astype(np.int16)
    
    return df

In [ ]:
months = train['date_block_num'].unique()

In [ ]:
cartesian = []
for month in months:
    shops_in_month = train.loc[train['date_block_num']==month, 'shop_id'].unique()
    items_in_month = train.loc[train['date_block_num']==month, 'item_id'].unique()
    cartesian.append(np.array(list(product(*[shops_in_month, items_in_month, [month]])), dtype='int32'))

In [ ]:
cartesian_df = pd.DataFrame(np.vstack(cartesian), columns = ['shop_id', 'item_id', 'date_block_num'], dtype=np.int32)

In [ ]:
cartesian_df.shape

**5. Агрегирование продаж до месячного уровня и отсечение целевой переменной**

In [ ]:
x = train.groupby(['shop_id', 'item_id', 'date_block_num'])['item_cnt_day'].sum().rename('item_cnt_month').reset_index()
x.head()

In [ ]:
x.shape

Теперь нужно объединить два фрейма данных
Cтолбцы, которые мы хотим объединить, являются пересечением shop_id, item_id и data_block_number

In [ ]:
new_train = pd.merge(cartesian_df, x, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)

По умолчанию pandas заполняет фреймы данных NaN. Вот почему мы используем fillna для замены всех NaN на ноль

In [ ]:
new_train['item_cnt_month'] = np.clip(new_train['item_cnt_month'], 0, 20)

Удаляем из памяти временные переменные

In [ ]:
del x
del cartesian_df
del cartesian
del cartesian_test
del cartesian_test_df
del feb
del jan
del items_test_list
del items_train_list
del train

In [ ]:
new_train.sort_values(['date_block_num','shop_id','item_id'], inplace = True)
new_train.head()

**6. Добавляем тестовый набор к тренировочному**

Во-первых, давайте вставим функцию data_block_num для тестового набора! Используя метод вставки панд, чтобы поместить этот новый столбец в определенный индекс. Это позволит нам легко связать тестовый набор с обучающим набором, прежде чем мы создадим средние кодировки и функции запаздывания

In [ ]:
test.insert(loc=3, column='date_block_num', value=34)

In [ ]:
test['item_cnt_month'] = 0

In [ ]:
test.head()

In [ ]:
new_train = new_train.append(test.drop('ID', axis = 1))

Объединение магазинов, товаров, категорий фреймов данных для добавления метки города, category_id, основной категории и подкатегории

In [ ]:
new_train = pd.merge(new_train, shops, on=['shop_id'], how='left')
new_train.head()

In [ ]:
new_train = pd.merge(new_train, items.drop('item_name', axis = 1), on=['item_id'], how='left')
new_train.head()

In [ ]:
new_train = pd.merge(new_train, categories.drop('item_category_name', axis = 1), on=['item_category_id'], how='left')
new_train.head()

**Generating Lag Features and Mean-Encodings**

In [ ]:
def generate_lag(train, months, lag_column):
    for month in months:
        # Speed up by grabbing only the useful bits
        train_shift = train[['date_block_num', 'shop_id', 'item_id', lag_column]].copy()
        train_shift.columns = ['date_block_num', 'shop_id', 'item_id', lag_column+'_lag_'+ str(month)]
        train_shift['date_block_num'] += month
        train = pd.merge(train, train_shift, on=['date_block_num', 'shop_id', 'item_id'], how='left')
    return train

In [ ]:
del items
del categories
del shops
del test

In [ ]:
new_train = downcast_dtypes(new_train)

In [ ]:
import gc  # сборщик мусора
gc.collect()

**Лаг для целевой переменной**

In [ ]:
%%time
new_train = generate_lag(new_train, [1,2,3,4,5,6,12], 'item_cnt_month')

**Лаг для месячного элемента-целевое среднее значение**

In [ ]:
%%time
group = new_train.groupby(['date_block_num', 'item_id'])['item_cnt_month'].mean().rename('item_month_mean').reset_index()
new_train = pd.merge(new_train, group, on=['date_block_num', 'item_id'], how='left')
new_train = generate_lag(new_train, [1,2,3,6,12], 'item_month_mean')
new_train.drop(['item_month_mean'], axis=1, inplace=True)

**Лаг для месячного магазина - целевое среднее значение**

In [ ]:
%%time
group = new_train.groupby(['date_block_num', 'shop_id'])['item_cnt_month'].mean().rename('shop_month_mean').reset_index()
new_train = pd.merge(new_train, group, on=['date_block_num', 'shop_id'], how='left')
new_train = generate_lag(new_train, [1,2,3,6,12], 'shop_month_mean')
new_train.drop(['shop_month_mean'], axis=1, inplace=True)

**Лаг для ежемесячного среднего значения по магазинам-категориям**

In [ ]:
%%time
group = new_train.groupby(['date_block_num', 'shop_id', 'item_category_id'])['item_cnt_month'].mean().rename('shop_category_month_mean').reset_index()
new_train = pd.merge(new_train, group, on=['date_block_num', 'shop_id', 'item_category_id'], how='left')
new_train = generate_lag(new_train, [1, 2], 'shop_category_month_mean')
new_train.drop(['shop_category_month_mean'], axis=1, inplace=True)

**Лаг для месячного среднего значения основной категории**

In [ ]:
%%time
group = new_train.groupby(['date_block_num', 'main_category_id'])['item_cnt_month'].mean().rename('main_category_month_mean').reset_index()
new_train = pd.merge(new_train, group, on=['date_block_num', 'main_category_id'], how='left')

new_train = generate_lag(new_train, [1], 'main_category_month_mean')
new_train.drop(['main_category_month_mean'], axis=1, inplace=True)

**Лаг для среднемесячного значения по подкатегориям**

In [ ]:
%%time
group = new_train.groupby(['date_block_num', 'sub_category_id'])['item_cnt_month'].mean().rename('sub_category_month_mean').reset_index()
new_train = pd.merge(new_train, group, on=['date_block_num', 'sub_category_id'], how='left')

new_train = generate_lag(new_train, [1], 'sub_category_month_mean')
new_train.drop(['sub_category_month_mean'], axis=1, inplace=True)

In [ ]:
new_train.tail()

**Создаем прогнозный месяц**

In [ ]:
new_train['month'] = new_train['date_block_num'] % 12

In [ ]:
holiday_dict = {
    0: 6,
    1: 3,
    2: 2,
    3: 8,
    4: 3,
    5: 3,
    6: 2,
    7: 8,
    8: 4,
    9: 8,
    10: 5,
    11: 4,
}

In [ ]:
new_train['holidays_in_month'] = new_train['month'].map(holiday_dict)

In [ ]:
new_train = downcast_dtypes(new_train)

# Моделирование

Особенности наших данных:
- данные хорошо структурированы и их много
- необходимо найти идеальную комбинацию направленную на оптимизацию ПО и железа для получения точных результатов за короткое время с минимальным использованием вычислительных ресурсов


In [ ]:
import lightgbm as lgb
import xgboost as xgb

In [ ]:
new_train = new_train[new_train.date_block_num > 11]
#X_train = new_train[new_train.date_block_num < 33].drop(['item_cnt_month'], axis=1)
#Y_train = new_train[new_train.date_block_num < 33]['item_cnt_month']

#X_valid = new_train[new_train.date_block_num == 33].drop(['item_cnt_month'], axis=1)
#Y_valid = new_train[new_train.date_block_num == 33]['item_cnt_month']

#X_test = new_train[new_train.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [ ]:
import gc
gc.collect()

In [ ]:
def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            df[col].fillna(0, inplace=True)         
    return df

new_train = fill_na(new_train)

In [ ]:
def lgbtrain():
    regressor1 = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.01, n_estimators=5000,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
    
    regressor_1 = regressor1.fit(new_train[new_train.date_block_num < 33].drop(['item_cnt_month'], axis=1).values, 
                               new_train[new_train.date_block_num < 33]['item_cnt_month'].values, 
                               eval_metric = 'rmse', 
                               eval_set = [(new_train[new_train.date_block_num < 33].drop(['item_cnt_month'], axis=1).values, 
                                            new_train[new_train.date_block_num < 33]['item_cnt_month'].values), 
                                           (new_train[new_train.date_block_num == 33].drop(['item_cnt_month'], axis=1).values, 
                                            new_train[new_train.date_block_num == 33]['item_cnt_month'].values)
                                          ], 
                               verbose=True,
                               early_stopping_rounds = 50,
                              )
    return regressor_1

In [ ]:
%%time
regressor_1 = lgbtrain()

In [ ]:
predictions1 = regressor_1.predict(new_train[new_train.date_block_num == 34].drop(['item_cnt_month'], axis = 1).values)

In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 11.7,8.27

cols = new_train.drop('item_cnt_month', axis = 1).columns
plt.barh(cols, regressor_1.feature_importances_)
plt.show()

In [ ]:
def xgtrain():
    regressor = xgb.XGBRegressor(n_estimators = 5000,
                                 learning_rate = 0.01,
                                 max_depth = 10,
                                 subsample = 0.5,
                                 colsample_bytree = 0.5)
    
    regressor_ = regressor.fit(new_train[new_train.date_block_num < 33].drop(['item_cnt_month'], axis=1).values, 
                               new_train[new_train.date_block_num < 33]['item_cnt_month'].values, 
                               eval_metric = 'rmse', 
                               eval_set = [(new_train[new_train.date_block_num < 33].drop(['item_cnt_month'], axis=1).values, 
                                            new_train[new_train.date_block_num < 33]['item_cnt_month'].values), 
                                           (new_train[new_train.date_block_num == 33].drop(['item_cnt_month'], axis=1).values, 
                                            new_train[new_train.date_block_num == 33]['item_cnt_month'].values)
                                          ], 
                               verbose=True,
                               early_stopping_rounds = 50,
                              )
    return regressor_

In [ ]:
%%time
regressor_ = xgtrain()

In [ ]:
predictions = regressor_.predict(new_train[new_train.date_block_num == 34].drop(['item_cnt_month'], axis = 1).values)

In [ ]:
rcParams['figure.figsize'] = 11.7,8.27

cols = new_train.drop('item_cnt_month', axis = 1).columns
plt.barh(cols, regressor_.feature_importances_)
plt.show()

Модели показали хорошие результаты (lgb = 0,96, xgb = 0,928), но XGBoost - король среди алгоритмов по решению задач регрессии.

Высокая результативность алгоритма, т.к.:
- процессы идут все параллельно
- использование отсечения деревьев
- аппаратная оптимизация

*Улучшения алгоритма:*
1. Регуляризация: Он штрафует сложные модели, используя как регуляризацию LASSO (L1), так и Ridge-регуляризацию (L2), для того, чтобы избежать переобучения.
2. Работа с разреженными данными: Алгоритм упрощает работу с разреженными данными, в процессе обучения заполняя пропущенные значения в зависимости от значения потерь. К тому же, он позволяет работать с различными узорами разреженности.
3. Метод взвешенных квантилей: XGBoost использует его для того, чтобы наиболее эффективно находить оптимальные точки разделения в случае работы со взвешенным датасетом.
4. Кросс-валидация: Алгоритм использует свой собственный метод кросс-валидации на каждой итерации. То есть, нам не нужно отдельно программировать этот поиск и определять количество итераций бустинга для каждого запуска.

In [ ]:
submission['item_cnt_month'] = predictions

In [ ]:
submission.to_csv('sales_faster_learn.csv', index=False)

In [ ]:
from IPython.display import FileLinks
FileLinks('.')